주식 종가 예측 : 
https://dacon.io/competitions/official/235800/data

1.Stock_List.csv : 종목 번호 데이터

 

종목명 : 주식 종목 명  
종목코드 : 주식 종목 코드번호  
상장시장 : 주식 종목 상장 시장 (KOSPI or KOSDAQ)

In [9]:
# !pip install bs4

Active code page: 65001
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=65601d2679600ccd40a327fa1f2a1cea9e481d21c3c1b834b9bbe0b2f69aaee0
  Stored in directory: c:\users\htaur\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


You should consider upgrading via the 'c:\users\htaur\anaconda3\envs\py37\python.exe -m pip install --upgrade pip' command.


In [4]:
# !pip install finance-datareader

Active code page: 65001


You should consider upgrading via the 'c:\users\htaur\anaconda3\envs\py37\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import os
import FinanceDataReader as fdr

In [3]:
path = './open'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
371,더네이쳐홀딩스,298540,KOSDAQ
372,코엔텍,029960,KOSDAQ
373,원익홀딩스,030530,KOSDAQ
374,웹케시,053580,KOSDAQ


In [4]:
start_date = '20200101'
end_date = '20201231'
sample_code = '005930' #삼성전자
stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,55500,56000,55000,55200,12993228,-0.010753
2020-01-03,56000,56600,54900,55500,15422255,0.005435
2020-01-06,54900,55600,54600,55500,10278951,0.000000
2020-01-07,55700,56400,55600,55800,10009778,0.005405
2020-01-08,56200,57400,55900,56800,23501171,0.017921
...,...,...,...,...,...,...
2020-12-23,72400,74000,72300,73900,19411326,0.022130
2020-12-24,74100,78800,74000,77800,32502870,0.052774
2020-12-28,79000,80100,78200,78700,40085044,0.011568


In [6]:
start_date = '20210104'
end_date = '20210910'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V') # 몇번째 주
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date']) #영업일 가져오기

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 36
HOW MANY "Business_days" : (180, 1)


In [11]:
data = fdr.DataReader('005930', start = start_date, end = end_date)[['Close']].reset_index()

,Date,Close
0,2021-01-04,83000
1,2021-01-05,83900
2,2021-01-06,82200
3,2021-01-07,82900
4,2021-01-08,88800
...,...,...
169,2021-09-06,77300
170,2021-09-07,76100
171,2021-09-08,76300
172,2021-09-09,75300


In [12]:
data = pd.merge(Business_days, data, how = 'outer')

In [15]:
data['weekday'] = data.Date.apply(lambda x : x.weekday())
data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
data.Close = data.Close.ffill()

In [16]:
data.head(10)

,Date,Close,weekday,weeknum
0,2021-01-04,83000.0,0,01
1,2021-01-05,83900.0,1,01
2,2021-01-06,82200.0,2,01
3,2021-01-07,82900.0,3,01
4,2021-01-08,88800.0,4,01
5,2021-01-11,91000.0,0,02
6,2021-01-12,90600.0,1,02
7,2021-01-13,89700.0,2,02
8,2021-01-14,89700.0,3,02
9,2021-01-15,88000.0,4,02


In [18]:
# 이동평균
data['Close'].ewm(5).mean()

0      83000.000000
1      83490.909091
2      82980.219780
3      82954.396423
4      84583.272415
           ...     
175    76035.145442
176    76045.954535
177    76088.295446
178    75956.912872
179    75847.427393
Name: Close, Length: 180, dtype: float64

In [17]:
data['ma5']=data['Close'].ewm(5).mean()
data['ma20']=data['Close'].ewm(20).mean()